In [1]:
import os, sys
os.environ["SPARK_HOME"] = '/usr/hdp/current/spark-client'
sys.path.insert(0, os.path.join(os.environ["SPARK_HOME"], 'python'))

# import package
import pyspark
from pyspark.context import SparkContext, SparkConf
from pyspark.sql import SQLContext

conf = SparkConf()
conf.set("spark.ui.port", 9568)
sc = SparkContext(conf=conf)
sqlContext =SQLContext(sc)

In [2]:
from pyspark import SparkContext
from pyspark import sql
from pyspark.sql import SQLContext
from pyspark.sql.functions import avg
import re
import nltk
from nltk.corpus import stopwords

In [ ]:
#loading data files
review = pd.read_csv('yelp_review.csv')
business = pd.read_csv('yelp_business.csv')

In [ ]:
#Removing non-food or restaurant businesses
business = business[map(lambda x: "Restaurant" in x or "Food" in x,business['categories'])]

In [ ]:
#Sanitizing 'Text' Field
review['text'] = review['text'].replace('\n',' ', regex=True)

In [ ]:
#Joining Review to Business table
combined = business.merge(review, on='business_id', how='inner', sort=False)
combined['stars_business']=combined['stars_x']
combined['stars_review']=combined['stars_y']
combined = combined[['business_id','name','stars_business','categories','review_id','stars_review','text','useful','funny','cool']]

In [ ]:
#Writing to CSV
combined.to_csv('yelp_combined_ns.csv',index = False, sep='^',encoding='utf-8')

In [3]:
sw = list(set(stopwords.words('english')))

str1 = str(' '.join(sw))
str2 = 'is am are was were have has had and the is has ive 1 2 3 4 5 6 7 8 9 0‘ourselves’, ‘hers’, ‘between’, ‘yourself’, ‘but’, ‘again’, ‘there’, ‘about’, ‘once’, ‘during’, ‘out’, ‘very’, ‘having’, ‘with’, ‘they’, ‘own’, ‘an’, ‘be’, ‘some’, ‘for’, ‘do’, ‘its’, ‘yours’, ‘such’, ‘into’, ‘of’, ‘most’, ‘itself’, ‘other’, ‘off’, ‘is’, ‘s’, ‘am’, ‘or’, ‘who’, ‘as’, ‘from’, ‘him’, ‘each’, ‘the’, ‘themselves’, ‘until’, ‘below’, ‘are’, ‘we’, ‘these’, ‘your’, ‘his’, ‘through’, ‘don’, ‘nor’, ‘me’, ‘were’, ‘her’, ‘more’, ‘himself’, ‘this’, ‘down’, ‘should’, ‘our’, ‘their’, ‘while’, ‘above’, ‘both’, ‘up’, ‘to’, ‘ours’, ‘had’, ‘she’, ‘all’, ‘no’, ‘when’, ‘at’, ‘any’, ‘before’, ‘them’, ‘same’, ‘and’, ‘been’, ‘have’, ‘in’, ‘will’, ‘on’, ‘does’, ‘yourselves’, ‘then’, ‘that’, ‘because’, ‘what’, ‘over’, ‘why’, ‘so’, ‘can’, ‘did’, ‘not’, ‘now’, ‘under’, ‘he’, ‘you’, ‘herself’, ‘has’, ‘just’, ‘where’, ‘too’, ‘only’, ‘myself’, ‘which’, ‘those’, ‘i’, ‘after’, ‘few’, ‘whom’, ‘t’, ‘being’, ‘if’, ‘theirs’, ‘my’, ‘against’, ‘a’, ‘by’, ‘doing’, ‘it’, ‘how’, ‘further’, ‘was’, ‘here’, ‘than’'
str1

'all just being over both through yourselves its before o hadn herself ll had should to only won under ours has do them his very they not during now him nor d did didn this she each further where few because doing some hasn are our ourselves out what for while re does above between mustn t be we who were here shouldn hers by on about couldn of against s isn or own into yourself down mightn wasn your from her their aren there been whom too wouldn themselves weren was until more himself that but don with than those he me myself ma these up will below ain can theirs my and ve then is am it doesn an as itself at have in any if again no when same how other which you shan needn haven after most such why a off i m yours so y the having once'

In [4]:
def review_category(s1,s2,s3):

    s1=s1.encode("utf-8")
    s2=s2.encode("utf-8")
    s3=s3.encode("utf-8")
    s1=s1.strip()
    s2=s2.strip()
    s3=s3.strip()
    
    if s1 == 'useful' : 
        return u'useful'
    elif int(s1) > int(s2) and int(s1) > int(s3) :
        return 'useful'
    elif int(s2) > int(s1) and int(s2) > int(s3) :
        return 'funny'
    elif int(s3) > int(s2) and int(s3) > int(s1) :
        return 'cool'
    else:
        return 'none'

    
#    s = re.sub(r'\\n',r'', re.sub(r'\\x[0-9A-Fa-f]+',r'', s))
#    s = (re.sub(r'@','',s)).replace('RT','')
#    s = ((s.lstrip('b\'')).replace('\'', '')).replace('"', '')
#    return ((s.split(' ')))


def string_len(s):

    s=s.encode("utf-8")
    s= s.strip()
    s= s.lower()
    s = re.sub(r'\\n',r'', re.sub(r'\\x[0-9A-Fa-f]+',r'', s))
    s = (re.sub(r'@','',s)).replace('RT','')
    s = ((s.lstrip('b\'')).replace('\'', '')).replace('"', '').replace(' .','')
    s = (s.split(' '))
    final = []
    for each in  s:
        if each != '' :
            if each not in str1 :
                if each not in str2:
                    final.append(each)
    return final


.filter(lambda x: "funny" in x[0][0])\

In [5]:
lines=sc.textFile("hdfs:///data/MSA_8050_Spring_18/proj08/yelp_combined_ns.csv" , use_unicode = 'False').map(lambda line: line.split("^"))\
            .map(lambda line: (review_category(line[7],line[8],line[9]),string_len(line[6])))\
            .flatMap(lambda x: [((x[0],str(i)), 1) for i in x[1] ] )\
            .filter(lambda x: "none" not in x[0][0])

#table2 = table1.filter(lambda x: "TEXT" in x[12])
lines.take(5000)

lines1=sc.parallelize(lines.take(50000))
print(lines1)

ParallelCollectionRDD[4] at parallelize at PythonRDD.scala:423


In [6]:
split3 = lines1.reduceByKey(lambda accum, n: accum + n)

In [13]:
split5 = split3.map(lambda x : (x[0][0],x[0][1])).sortBy(lambda x: (x[0][0]), ascending = False)
split6 = split5.reduceByKey(lambda a,b : a+','+str(b)).sortBy(lambda x: (x[0]), ascending = True)

In [10]:
for i in split6:
    str1 = i[1]
    str1.split(',')
    b = (str1.split(',')[0:15])
    final = []
    for each in  b:
        if each != '':
            final.append(each)
    print(str(i[0]) + " category: " + ' '.join(final))

cool catogery: made tortillas right lived mileage slice corndogs someone fee attended spicy cons cabinets used milestones
funny catogery: response switching meal complain find appears appreciated arm side correct 815 asked thinks options oily
useful catogery: picadillo 20s pastas par ironically adorned television 1320 ceiling warmed shut begin lightbulbs non notice


Top 10 words for different Rating

In [ ]:
import os, sys
os.environ["SPARK_HOME"] = '/usr/hdp/current/spark-client'
sys.path.insert(0, os.path.join(os.environ["SPARK_HOME"], 'python'))

# import package
import pyspark
from pyspark.context import SparkContext, SparkConf
import string
conf = SparkConf()
conf.set("spark.ui.port", 9564)
sc = SparkContext(conf=conf)


In [ ]:
rawdata = sc.textFile("/data/MSA_8050_Spring_18/proj08/yelp_combined.csv").map(lambda x: x.split('^'))
header=rawdata.first()
rawdata1=(rawdata.filter(lambda x: x!= header))

In [ ]:
from nltk.corpus import stopwords

stopwords_en = []
for word in stopwords.words('english'):
    stopwords_en = stopwords_en + [unicodedata.normalize('NFKD', word).encode('ascii','ignore')]
stopwords_en

def stopwordremover(query):
    querywords = query.split()
    resultwords  = [word for word in querywords if word.lower() not in stopwords_en]
    result = ' '.join(resultwords)
    return result

In [ ]:
#creating a dictionary to remove punctuation from unicode strings
remove_punctuation_map = {}

for word in string.punctuation:
    remove_punctuation_map[ord(word)]=None

In [ ]:
#creating new list element without punctuation
rawdata1 = rawdata1.map(lambda x: x + [x[5].translate(remove_punctuation_map).lower()])

In [ ]:
rawdata1 = rawdata1.map(lambda x: x + [stopwordremover(unicodedata.normalize('NFKD', x[9]).encode('ascii','ignore'))])

In [ ]:

keyvaluepair2 = rawdata1.map(lambda x: (float(x[3]), x[6].encode('utf-8').split()))
keyvaluepair3 = keyvaluepair2.flatMapValues(lambda x: x)
keyvaluepair4 = keyvaluepair3.map(lambda x: ((x[0], x[1]),1))
#kvp4= sc.parallelize(keyvaluepair4.take(16000))

keyvaluepair5 = keyvaluepair4.reduceByKey(lambda x, y: x + y)

In [ ]:

keyvaluepair5.take(5)



#Creating key, value pair where key = star rating and value is (Count of word with respect to star rating, word)
keyvaluepair6 = keyvaluepair5.map(lambda x: (x[0][0], (x[1], x[0][1])))

In [ ]:

#Sorting the values and taking the top 10 words based on frequency
keyvaluepair7 = keyvaluepair6.groupByKey().mapValues(lambda x: list(x)).map(lambda x: (x[0], sorted(x[1], reverse=True)[0:10]))                             .sortByKey() 



In [ ]:
#Printing the results
print 'Top 10 common words'      + '\n' + str(keyvaluepair7.collect()[0][0]) + ' star rating :  ' + str(keyvaluepair7.collect()[0][1]) 
+ '\n' + str(keyvaluepair7.collect()[1][0]) + ' star rating :  ' + str(keyvaluepair7.collect()[1][1])
+ '\n' + str(keyvaluepair7.collect()[2][0]) + ' star rating :  ' + str(keyvaluepair7.collect()[2][1]) 
+ '\n' + str(keyvaluepair7.collect()[3][0]) + ' star rating :  ' + str(keyvaluepair7.collect()[3][1])
+ '\n' + str(keyvaluepair7.collect()[4][0]) + ' star rating :  ' + str(keyvaluepair7.collect()[4][1]) 


In [ ]:
###SENTIMENT ANALYSIS

In [ ]:
Italian_wordcount = results[map(lambda x: "Italian" in x,combined['categories'])]
American_wordcount = results[map(lambda x: "American" in x,combined['categories'])]
Mexican_wordcount = results[map(lambda x: "Mexican" in x,combined['categories'])]
Chinese_wordcount = results[map(lambda x: "Chinese" in x,combined['categories'])]
Indian_wordcount = results[map(lambda x: "Indian" in x,combined['categories'])]
#Get the wordcounts by restaurant category

In [ ]:
positive = open('positive-words.txt').read().split()
negative = open('negative-words.txt').read().split()
#load positive and negative words

In [ ]:
Italian_wordcount = pd.DataFrame(Italian_wordcount,columns=['count'])
American_wordcount = pd.DataFrame(American_wordcount,columns=['count'])
Mexican_wordcount = pd.DataFrame(Mexican_wordcount,columns=['count'])
Chinese_wordcount = pd.DataFrame(Chinese_wordcount,columns=['count'])
Indian_wordcount = pd.DataFrame(Indian_wordcount,columns=['count'])
#subset wordcount by restaurant time

In [ ]:
print "Italian"
print np.sum(Italian_wordcount[Italian_wordcount.index.isin(positive)]['count'])
print "American"
print np.sum(American_wordcount[American_wordcount.index.isin(positive)])
print "Mexican"
print np.sum(Mexican_wordcount[Mexican_wordcount.index.isin(positive)])
print "Chinese"
print np.sum(Chinese_wordcount[Chinese_wordcount.index.isin(positive)])
print "Indian"
print np.sum(Indian_wordcount[Indian_wordcount.index.isin(positive)])

In [ ]:
print "Italian"
print np.sum(Italian_wordcount[Italian_wordcount.index.isin(negative)])
print "American"
print np.sum(American_wordcount[American_wordcount.index.isin(negative)])
print "Mexican"
print np.sum(Mexican_wordcount[Mexican_wordcount.index.isin(negative)])
print "Chinese"
print np.sum(Chinese_wordcount[Chinese_wordcount.index.isin(negative)])
print "Indian"
print np.sum(Indian_wordcount[Indian_wordcount.index.isin(negative)])

In [ ]:
#cool words overall
cool_wordcount = cool.text.str.split(expand=True).stack().value_counts()
cool_wordcount = pd.DataFrame(cool_wordcount,columns=['count'])
cool_positive = cool_wordcount[cool_wordcount.index.isin(list_positive)]

In [ ]:
#number of positive words for cool
cool_positive['count'].sum()

In [ ]:
#negative words for cool
cool_negative = cool_wordcount[cool_wordcount.index.isin(list_negative)]

In [ ]:
#number of negative words for cool
cool_negative['count'].sum()

In [ ]:
#useful words overall
useful_wordcount = useful.text.str.split(expand=True).stack().value_counts()
useful_wordcount = pd.DataFrame(useful_wordcount,columns=['count'])
useful_positive = useful_wordcount[useful_wordcount.index.isin(list_positive)]

In [ ]:
#number of positive words for useful
useful_positive['count'].sum()

In [ ]:
#negative words for useful
useful_negative = useful_wordcount[useful_wordcount.index.isin(list_negative)]

In [ ]:
#number of negative words for useful
useful_negative['count'].sum()

In [ ]:
#funny words overall
funny_wordcount = funny.text.str.split(expand=True).stack().value_counts()
funny_wordcount = pd.DataFrame(funny_wordcount,columns=['count'])
funny_positive = funny_wordcount[funny_wordcount.index.isin(list_positive)]

In [ ]:
#number of positive words for funny
funny_positive['count'].sum()

In [ ]:
#negative words for funny
funny_negative = funny_wordcount[funny_wordcount.index.isin(list_negative)]

In [ ]:
#number of negative words for funny
funny_negative['count'].sum()

In [ ]:
#average length of cool, useful, and funny
cool_wc=cool['text'].str.split().str.len().mean()
useful_wc=useful['text'].str.split().str.len().mean()
funny_wc=funny['text'].str.split().str.len().mean()

In [ ]:
#top 10 most common words overall
print Counter(" ".join(yelp_review["text"]).split()).most_common(10)

In [ ]:
#average word count overall
overall_wc=yelp_review['text'].str.split().str.len().mean()
print "Average Word Count is: ", overall_wc

In [ ]:
#separate data into star ratings
star1 = yelp_review[yelp_review['stars_review']==1.]
star2 = yelp_review[yelp_review['stars_review']==2.]
star3 = yelp_review[yelp_review['stars_review']==3.]
star4 = yelp_review[yelp_review['stars_review']==4.]
star5 = yelp_review[yelp_review['stars_review']==5.]

In [ ]:
#top 10 words for 1 star
Counter(" ".join(star1["text"]).split()).most_common(10)

In [ ]:
#top 10 words for 2 star
Counter(" ".join(star2["text"]).split()).most_common(10)

In [ ]:
#top 10 words for 3 star
Counter(" ".join(star3["text"]).split()).most_common(10)

In [ ]:
#top 10 words for 4 star
Counter(" ".join(star4["text"]).split()).most_common(10)

In [ ]:
#top 10 words for 5 star
Counter(" ".join(star5["text"]).split()).most_common(10)

In [ ]:
#average word count by star rating
star1_avg_wc=star1['text'].str.split().str.len().mean()
star2_avg_wc=star2['text'].str.split().str.len().mean()
star3_avg_wc=star3['text'].str.split().str.len().mean()
star4_avg_wc=star4['text'].str.split().str.len().mean()
star5_avg_wc=star5['text'].str.split().str.len().mean()

print "1 Star Average Word Count is:", star1_avg_wc
print "2 Star Average Word Count is:", star2_avg_wc
print "3 Star Average Word Count is:", star3_avg_wc
print "4 Star Average Word Count is:", star4_avg_wc
print "5 Star Average Word Count is:", star5_avg_wc

In [ ]:
#lexicon of positive words/sentiments
list_positive = open("positive-words.txt").read().split()
list_positive

In [ ]:
#lexicon of negative words/sentiments
list_negative = open("negative-words.txt").read().split()
list_negative

In [ ]:
#positive words overall
overall_wordcount = yelp_review.text.str.split(expand=True).stack().value_counts()
overall_wordcount = pd.DataFrame(overall_wordcount,columns=['count'])
overall_positive = overall_wordcount[overall_wordcount.index.isin(list_positive)]

In [ ]:
#number of positive words overall
overall_positive['count'].sum()

In [ ]:
#negative words overall
overall_negative = overall_wordcount[overall_wordcount.index.isin(list_negative)]

In [ ]:
#number of negative words overall
overall_negative['count'].sum()

In [ ]:
#positive words onestar
onestar_wordcount = star1.text.str.split(expand=True).stack().value_counts()
onestar_wordcount = pd.DataFrame(onestar_wordcount,columns=['count'])
onestar_positive = onestar_wordcount[onestar_wordcount.index.isin(list_positive)]

In [ ]:
#number of positive words onestar
onestar_positive['count'].sum()

In [ ]:
#negative words onestar
onestar_negative = onestar_wordcount[onestar_wordcount.index.isin(list_negative)]

In [ ]:
#number of negative words onestar
onestar_negative['count'].sum()

In [ ]:
#positive words twostar
twostar_wordcount = star2.text.str.split(expand=True).stack().value_counts()
twostar_wordcount = pd.DataFrame(twostar_wordcount,columns=['count'])
twostar_positive = twostar_wordcount[twostar_wordcount.index.isin(list_positive)]

In [ ]:
#number of positive words twostar
twostar_positive['count'].sum()

In [ ]:
#negative words twostar
twostar_negative = twostar_wordcount[twostar_wordcount.index.isin(list_negative)]

In [ ]:
#number of negative words twostar
twostar_negative['count'].sum()

In [ ]:
#positive words threestar
threestar_wordcount = star3.text.str.split(expand=True).stack().value_counts()
threestar_wordcount = pd.DataFrame(threestar_wordcount,columns=['count'])
threestar_positive = threestar_wordcount[threestar_wordcount.index.isin(list_positive)]

In [ ]:
#number of positive words threestar
threestar_positive['count'].sum()

In [ ]:
#negative words threestar
threestar_negative = threestar_wordcount[threestar_wordcount.index.isin(list_negative)]

In [ ]:
#number of negative words threestar
threestar_negative['count'].sum()

In [ ]:
#positive words fourstar
fourstar_wordcount = star4.text.str.split(expand=True).stack().value_counts()
fourstar_wordcount = pd.DataFrame(fourstar_wordcount,columns=['count'])
fourstar_positive = fourstar_wordcount[fourstar_wordcount.index.isin(list_positive)]

In [ ]:
#number of positive words fourstar
fourstar_positive['count'].sum()

In [ ]:
#negative words fourstar
fourstar_negative = fourstar_wordcount[fourstar_wordcount.index.isin(list_negative)]

In [ ]:
#number of negative words fourstar
fourstar_negative['count'].sum()

In [ ]:
#positive words for fivestar
fivestar_wordcount = star5.text.str.split(expand=True).stack().value_counts()
fivestar_wordcount = pd.DataFrame(fivestar_wordcount,columns=['count'])
fivestar_positive = fivestar_wordcount[fivestar_wordcount.index.isin(list_positive)]

In [ ]:
#number of positive words fivestar
fivestar_positive['count'].sum()

In [ ]:
#negative words fivestar
fivestar_negative = fivestar_wordcount[fivestar_wordcount.index.isin(list_negative)]

In [ ]:
#number of negative word fivestar
fivestar_negative['count'].sum()